In [64]:
import sys, os, numpy as np, pytest
sys.path.append(os.path.abspath(".."))

from src.pricing_black_scholes import BlackScholesOption
from src.pricing_binomial import BinomialOption
from src.pricing_montecarlo import MonteCarloOption

#### Princing black scholes

In [65]:
# Test: Price call y put
def test_call_option_price():
    option = BlackScholesOption(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="call")
    price = option.price()
    print(f"Call price: {price:.4f}")
    assert round(price, 2) == 10.45


def test_put_option_price():
    option = BlackScholesOption(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="put")
    price = option.price()
    print(f"Put price: {price:.4f}")
    assert round(price, 2) == 5.57

# Test: Greeks
def test_greeks_call():
    option = BlackScholesOption(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="call")
    greeks = option.greeks()
    print("Call Greeks:", greeks)
    assert "delta" in greeks and "gamma" in greeks and "vega" in greeks and "theta" in greeks and "rho" in greeks


def test_greeks_put():
    option = BlackScholesOption(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type="put")
    greeks = option.greeks()
    print("Put Greeks:", greeks)
    assert "delta" in greeks and "gamma" in greeks and "vega" in greeks and "theta" in greeks and "rho" in greeks


# Test: Implied volatility
def test_implied_volatility():
    market_price = 10.45
    iv = BlackScholesOption.implied_volatility_newton(
        market_price, S=100, K=100, T=1, r=0.05, option_type="call"
    )
    print(f"Implied volatility (call): {iv:.4f}")
    assert abs(iv - 0.2) < 0.01


# Test: Err Validation
def test_invalid_option_type():
    try:
        BlackScholesOption(100, 100, 1, 0.05, 0.2, option_type="banana")
        print("❌ Invalid option type did not raise error")
    except ValueError:
        print("✅ Invalid option type raises ValueError")


def test_invalid_inputs():
    for S, K, T, sigma in [(-100, 100, 1, 0.2), (100, 0, 1, 0.2), (100, 100, -1, 0.2), (100, 100, 1, -0.2)]:
        try:
            BlackScholesOption(S, K, T, 0.05, sigma, option_type="call")
            print(f"❌ Invalid input did not raise error for S={S}, K={K}, T={T}, sigma={sigma}")
        except ValueError:
            print(f"✅ Invalid input raises ValueError for S={S}, K={K}, T={T}, sigma={sigma}")


# Execute all tests and show output
tests = [
    ("call",  test_call_option_price),
    ("put",   test_put_option_price),
    ("greeks_call", test_greeks_call),
    ("greeks_put", test_greeks_put),
    ("implied_vol", test_implied_volatility),
    ("invalid_type", test_invalid_option_type),
    ("invalid_inputs", test_invalid_inputs),
]

for name, fn in tests:
    print(f"\nRunning: {name}")
    try:
        fn()
        print(f"{name}: ✅ OK")
    except AssertionError as e:
        print(f"{name}: ❌ {e}")


Running: call
Call price: 10.4506
call: ✅ OK

Running: put
Put price: 5.5735
put: ✅ OK

Running: greeks_call
Call Greeks: {'price': 10.450583572185565, 'delta': 0.6368306511756191, 'gamma': 0.018762017345846895, 'vega': 0.3752403469169379, 'theta': -0.01757267820941972, 'rho': 0.5323248154537634}
greeks_call: ✅ OK

Running: greeks_put
Put Greeks: {'price': 5.573526022256971, 'delta': -0.3631693488243809, 'gamma': 0.018762017345846895, 'vega': 0.3752403469169379, 'theta': -0.004542138147766099, 'rho': -0.4189046090469506}
greeks_put: ✅ OK

Running: implied_vol
Implied volatility (call): 0.2000
implied_vol: ✅ OK

Running: invalid_type
✅ Invalid option type raises ValueError
invalid_type: ✅ OK

Running: invalid_inputs
✅ Invalid input raises ValueError for S=-100, K=100, T=1, sigma=0.2
✅ Invalid input raises ValueError for S=100, K=0, T=1, sigma=0.2
✅ Invalid input raises ValueError for S=100, K=100, T=-1, sigma=0.2
✅ Invalid input raises ValueError for S=100, K=100, T=1, sigma=-0.2
inval

#### Princing Binomial

In [66]:
# Test: European call and put price
def test_binomial_european_call():
    option = BinomialOption(S=100, K=100, T=1, r=0.05, sigma=0.2, N=100, option_type="call")
    price = option.price_european()
    print(f"European Call price: {price:.4f}")
    assert price > 0


def test_binomial_european_put():
    option = BinomialOption(S=100, K=100, T=1, r=0.05, sigma=0.2, N=100, option_type="put")
    price = option.price_european()
    print(f"European Put price: {price:.4f}")
    assert price > 0


# Test: American call and put price
def test_binomial_american_call():
    option = BinomialOption(S=100, K=100, T=1, r=0.05, sigma=0.2, N=100, option_type="call")
    price = option.price_american()
    print(f"American Call price: {price:.4f}")
    assert price > 0


def test_binomial_american_put():
    option = BinomialOption(S=100, K=100, T=1, r=0.05, sigma=0.2, N=100, option_type="put")
    price = option.price_american()
    print(f"American Put price: {price:.4f}")
    assert price > 0


# Test: Price and tree output for European and American
def test_price_and_tree_european():
    option = BinomialOption(S=100, K=100, T=1, r=0.05, sigma=0.2, N=5, option_type="call")
    price, tree = option.price_and_tree(american=False)
    print("European price (with tree):", price)
    print("Asset tree (European):\n", tree)
    assert price > 0


def test_price_and_tree_american():
    option = BinomialOption(S=100, K=100, T=1, r=0.05, sigma=0.2, N=5, option_type="put")
    price, tree = option.price_and_tree(american=True)
    print("American price (with tree):", price)
    print("Asset tree (American):\n", tree)
    assert price > 0


# Test: Validation errors
def test_invalid_option_type():
    try:
        BinomialOption(100, 100, 1, 0.05, 0.2, 100, option_type="banana")
        print("❌ Invalid option type did not raise error")
    except ValueError:
        print("✅ Invalid option type raises ValueError")


def test_invalid_inputs():
    for S, K, T, sigma, N in [(-100, 100, 1, 0.2, 100), (100, 0, 1, 0.2, 100), (100, 100, -1, 0.2, 100), (100, 100, 1, -0.2, 100), (100, 100, 1, 0.2, 0)]:
        try:
            BinomialOption(S, K, T, 0.05, sigma, N, option_type="call")
            print(f"❌ Invalid input did not raise error for S={S}, K={K}, T={T}, sigma={sigma}, N={N}")
        except ValueError:
            print(f"✅ Invalid input raises ValueError for S={S}, K={K}, T={T}, sigma={sigma}, N={N}")

# Run all tests and show output
tests = [
    ("euro_call", test_binomial_european_call),
    ("euro_put", test_binomial_european_put),
    ("amer_call", test_binomial_american_call),
    ("amer_put", test_binomial_american_put),
    ("tree_euro", test_price_and_tree_european),
    ("tree_amer", test_price_and_tree_american),
    ("invalid_type", test_invalid_option_type),
    ("invalid_inputs", test_invalid_inputs),
]

for name, fn in tests:
    print(f"\nRunning: {name}")
    try:
        fn()
        print(f"{name}: ✅ OK")
    except AssertionError as e:
        print(f"{name}: ❌ {e}")


Running: euro_call
European Call price: 10.4306
euro_call: ✅ OK

Running: euro_put
European Put price: 5.5536
euro_put: ✅ OK

Running: amer_call
American Call price: 10.4306
amer_call: ✅ OK

Running: amer_put
American Put price: 6.0824
amer_put: ✅ OK

Running: tree_euro
European price (with tree): 10.805933920410084
Asset tree (European):
 [156.39483159 130.77762259 109.35646911  91.44406436  76.46568122
  63.94073192]
tree_euro: ✅ OK

Running: tree_amer
American price (with tree): 6.367829748451127
Asset tree (American):
 [[100.         109.35646911 119.58837337 130.77762259 143.01379046
  156.39483159]
 [  0.          91.44406436 100.         109.35646911 119.58837337
  130.77762259]
 [  0.           0.          83.62016907  91.44406436 100.
  109.35646911]
 [  0.           0.           0.          76.46568122  83.62016907
   91.44406436]
 [  0.           0.           0.           0.          69.92332675
   76.46568122]
 [  0.           0.           0.           0.           0.
   6

#### Princing Montecarlo

In [67]:
# Set a fixed random seed for reproducibility
np.random.seed(42)

# Common parameters for all tests
params = dict(S=100, K=100, T=1.0, r=0.05, sigma=0.2, option_type="call", n_simulations=5000, n_steps=50)



"""
Test: price_asian
This test checks the Asian option price using arithmetic average
"""

try:
    option = MonteCarloOption(**params)
    asian_price = option.price_asian()
    print(f"Asian option price (arithmetic average): {asian_price:.4f}")
    assert asian_price > 0
    print("price_asian: ✅ PASSED")
except Exception as e:
    print(f"price_asian: ❌ FAILED ({e})")



"""
Test: price_asian_geometric
This test checks the Asian option price using geometric average
"""

try:
    geo_price = option.price_asian_geometric()
    print(f"Asian option price (geometric average): {geo_price:.4f}")
    assert geo_price > 0
    print("price_asian_geometric: ✅ PASSED")
except Exception as e:
    print(f"price_asian_geometric: ❌ FAILED ({e})")



"""
Test: price_digital_barrier
This test checks the digital barrier option price for up-and-in and up-and-out
"""

try:
    digital_upin = option.price_digital_barrier(barrier=110, barrier_type="up-and-in", payout=1.0)
    print(f"Digital barrier (up-and-in) price: {digital_upin:.4f}")
    assert digital_upin >= 0
    digital_upout = option.price_digital_barrier(barrier=110, barrier_type="up-and-out", payout=1.0)
    print(f"Digital barrier (up-and-out) price: {digital_upout:.4f}")
    assert digital_upout >= 0
    print("price_digital_barrier: ✅ PASSED")
except Exception as e:
    print(f"price_digital_barrier: ❌ FAILED ({e})")



"""
Test: price_lookback
This test checks the lookback option price for both fixed and floating strike
"""

try:
    lookback_fixed = option.price_lookback(strike_type="fixed")
    print(f"Lookback option price (fixed strike): {lookback_fixed:.4f}")
    assert lookback_fixed > 0
    lookback_floating = option.price_lookback(strike_type="floating")
    print(f"Lookback option price (floating strike): {lookback_floating:.4f}")
    assert lookback_floating > 0
    print("price_lookback: ✅ PASSED")
except Exception as e:
    print(f"price_lookback: ❌ FAILED ({e})")



"""
Test: price_vanilla
This test checks the vanilla European option price
"""

try:
    vanilla_price = option.price_vanilla()
    print(f"Vanilla option price: {vanilla_price:.4f}")
    assert vanilla_price > 0
    print("price_vanilla: ✅ PASSED")
except Exception as e:
    print(f"price_vanilla: ❌ FAILED ({e})")



"""
Test: barrier_knock_in_out_payoff_paths
This test checks the knock-in and knock-out barrier payoff calculation
"""

try:
    knockin_payoff = option.barrier_knock_in_out_payoff_paths(barrier=110, barrier_type="up-and-in", payout=1.0)
    print(f"Knock-in barrier payoff: {knockin_payoff:.4f}")
    assert knockin_payoff >= 0
    knockout_payoff = option.barrier_knock_in_out_payoff_paths(barrier=110, barrier_type="up-and-out", payout=1.0)
    print(f"Knock-out barrier payoff: {knockout_payoff:.4f}")
    assert knockout_payoff >= 0
    print("barrier_knock_in_out_payoff_paths: ✅ PASSED")
except Exception as e:
    print(f"barrier_knock_in_out_payoff_paths: ❌ FAILED ({e})")



"""
Test: greek (delta, vega, rho, theta)
This test checks the calculation of Greeks using finite differences
"""

try:
    for greek in ["delta", "vega", "rho", "theta"]:
        value = option.greek(greek)
        print(f"Greek {greek}: {value:.4f}")
        assert np.isfinite(value)
    print("greek: ✅ PASSED")
except Exception as e:
    print(f"greek: ❌ FAILED ({e})")



"""
# Test: input validation (invalid option_type, negative inputs, invalid barrier/strike types)
"""

try:
    MonteCarloOption(100, 100, 1, 0.05, 0.2, option_type="banana")
    print("❌ Invalid option_type did not raise error")
except ValueError:
    print("✅ Invalid option_type raises ValueError")

try:
    MonteCarloOption(-100, 100, 1, 0.05, 0.2)
    print("❌ Negative S did not raise error")
except ValueError:
    print("✅ Negative S raises ValueError")

try:
    option.price_digital_barrier(barrier=110, barrier_type="banana")
    print("❌ Invalid barrier_type did not raise error")
except ValueError:
    print("✅ Invalid barrier_type raises ValueError")

try:
    option.price_lookback(strike_type="banana")
    print("❌ Invalid strike_type did not raise error")
except ValueError:
    print("✅ Invalid strike_type raises ValueError")

Asian option price (arithmetic average): 5.9365
price_asian: ✅ PASSED
Asian option price (geometric average): 5.5986
price_asian_geometric: ✅ PASSED
Digital barrier (up-and-in) price: 0.4802
Digital barrier (up-and-out) price: 0.0548
price_digital_barrier: ✅ PASSED
Lookback option price (fixed strike): 17.0569
Lookback option price (floating strike): 15.3855
price_lookback: ✅ PASSED
Vanilla option price: 10.4368
price_vanilla: ✅ PASSED
Knock-in barrier payoff: 0.4798
Knock-out barrier payoff: 0.0559
barrier_knock_in_out_payoff_paths: ✅ PASSED
Greek delta: 20.3489
Greek vega: 61.2959
Greek rho: 67.6272
Greek theta: 18.6381
greek: ✅ PASSED
✅ Invalid option_type raises ValueError
✅ Negative S raises ValueError
✅ Invalid barrier_type raises ValueError
✅ Invalid strike_type raises ValueError
